In [25]:

from __future__ import division
from keras.models import load_model
import cv2
import numpy as np

from random import choice
from math import sqrt
import random as rnd

In [26]:
REV_CLASS_MAP = {
    0: "rock",
    1: "paper",
    2: "scissors",
    3: "none"
}



def mapper(val):
    return REV_CLASS_MAP[val]

In [27]:
playerscore = 0
compscore = 0
tiescore = 0
def calculate_winner(move1, move2):
    global playerscore
    global compscore
    global tiescore
    if move1 == move2:
        tiescore = tiescore + 1
        return "Tie"

    if move1 == "rock":
        if move2 == "scissors":
            playerscore = playerscore + 1
            return "User"
        if move2 == "paper":
            compscore = compscore + 1
            return "Computer"

    if move1 == "paper":
        if move2 == "rock":
            playerscore = playerscore + 1
            return "User"
        if move2 == "scissors":
            compscore = compscore + 1
            return "Computer"

    if move1 == "scissors":
        if move2 == "paper":
            playerscore = playerscore + 1
            return "User"
        if move2 == "rock":
            compscore = compscore + 1
            return "Computer"

In [28]:
def search(pat, txt):
    M = len(pat)
    N = len(txt)

    flag=None

    # A loop to slide pat[] one by one */
    for i in range(N - M + 1):
        j = 0

        while(j < M):
            if (txt[i + j] != pat[j]):
                break
            j += 1

        if (j == M and i+3<N):
           
            flag=i

            
    if flag!=None:
        return txt[flag+3]



In [29]:
model = load_model("rock-paper-scissors-model.h5")

cap = cv2.VideoCapture(0)

cap.set(3, 1920)
cap.set(4, 1080)

prev_move = None

In [30]:
RPS_count = { '000' : 3, '001' : 3, '002' : 3, '010' : 3, '011' : 3, '012' : 3, '020' : 3, '021' : 3, '022' : 3, '100' : 3, '101' : 3, '102' : 3, '110' : 3, '111' : 3, '112' : 3, '120' : 3, '121' : 3, '122' : 3, '200' : 3, '201' : 3, '202' : 3, '210' : 3, '211' : 3, '212' : 3, '220' : 3, '221' : 3, '222' : 3 }

RPS_disp  = {'0' : 'rock', '1' : 'paper', '2' : 'scissor'}



last2 = '33'

history = ""
while True:
    ret, frame = cap.read()
    if not ret:
        continue

    # rectangle for user to play
    cv2.rectangle(frame, (100, 100), (500, 500), (255, 255, 255), 2)
    # rectangle for computer to play
    cv2.rectangle(frame, (800, 100), (1200, 500), (255, 255, 255), 2)

    # extract the region of image within the user rectangle
    roi = frame[100:500, 100:500]
    img = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (227, 227))

    # predict the move made
    pred = model.predict(np.array([img]))
    move_code = np.argmax(pred[0])
    user_move_name = mapper(move_code)

    # predict the winner (human vs computer)
    if prev_move != user_move_name:
        if user_move_name != "none":
            
            
            if user_move_name == "rock":
                x='0'
                
            if user_move_name == "paper":
                x='1'
                
            if user_move_name == "scissors":
                x='2'
                
       
          
            
            if(last2[0] == '3'):
                computer_move_name = choice(['rock', 'paper', 'scissors'])
                
            else:
                r_count = RPS_count[last2 + '0']
                p_count = RPS_count[last2 + '1']
                s_count = RPS_count[last2 + '2']
                
           
                tot_count = r_count + p_count + s_count
                
            
                q_dist = [ r_count/tot_count, p_count/tot_count, 1- (r_count/tot_count) - (p_count/tot_count) ]
            


                result = [ max(q_dist[2]-q_dist[1],0), max(q_dist[0]-q_dist[2],0), max(q_dist[1]-q_dist[0],0) ]
                
               
     
                resultnorm = sqrt(result[0]*result[0] + result[1]*result[1] + result[2]*result[2])
                result = [result[0]/resultnorm, result[1]/resultnorm, 1 - result[0]/resultnorm - result[1]/resultnorm]
                
                
                


                y = rnd.uniform(0,1)
                
                
      

                if y <= result[0]:
                    computer_move_name = "rock"
                elif y <= result[0] + result[1]:
                    computer_move_name = "paper"
                else:
                    computer_move_name = "scissors"
                
                
                print('computer_move_name 1 --- ',computer_move_name)
              

                
                #update dictionary
                RPS_count[last2+x] += 1
                
                        
                
            last2 = last2[1] + x
            
            print('last2 ',last2)
            
            
            
            print('history -----  ' + history)
            if len(history)>5:

                last_chars = history[-3:]

                print('last_chars ---------------- '+ last_chars)
                temp = search(last_chars, history)


                if temp != None:
                 
                    if temp=='0':
                        computer_move_name = "paper"
                    elif temp=='1':
                        computer_move_name = "scissors"
                    else:
                        computer_move_name = "rock"


            print('computer_move_name 2 --------',computer_move_name) 

            print('user_move_name 2 --------',user_move_name) 
           
            
            
            winner = calculate_winner(user_move_name, computer_move_name)
            history += x
        else:
            computer_move_name = "none"
            winner = "Waiting..."
    prev_move = user_move_name

    # display the information
    font = cv2.FONT_HERSHEY_SIMPLEX   
    
    cv2.putText(frame, "Player: " + str(playerscore),
                (100, 30), font, 1.2, (0, 0, 255), 2, cv2.LINE_AA)
    
    cv2.putText(frame, "Tie: " + str(tiescore),
                (450, 30), font, 1.2, (0, 0, 255), 2, cv2.LINE_AA)
    
    cv2.putText(frame, "Computer: " + str(compscore),
                (800, 30), font, 1.2, (0, 0, 255), 2, cv2.LINE_AA)
    
    
    
    
    cv2.putText(frame, "Your Move: " + user_move_name,
                (100, 70), font, 1.2, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.putText(frame, "Computer's Move: " + computer_move_name,
                (800, 70), font, 1.2, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.putText(frame, "Winner: " + str(winner),
                (400, 600), font, 2, (0, 0, 255), 4, cv2.LINE_AA)

    if computer_move_name != "none":
        icon = cv2.imread(
            "images/{}.png".format(computer_move_name))
        icon = cv2.resize(icon, (400, 400))
        frame[100:500, 800:1200] = icon

    cv2.imshow("Rock Paper Scissors", frame)

    k = cv2.waitKey(10)
    if k == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

last2  30
history -----  
computer_move_name 2 -------- paper
user_move_name 2 -------- rock
last2  01
history -----  0
computer_move_name 2 -------- scissors
user_move_name 2 -------- paper
y  0.6991686261497994
computer_move_name 1 ---  rock
last2  12
history -----  01
computer_move_name 2 -------- rock
user_move_name 2 -------- scissors
y  0.4326986727099066
computer_move_name 1 ---  rock
last2  20
history -----  012
computer_move_name 2 -------- rock
user_move_name 2 -------- rock
y  0.31423059182871693
computer_move_name 1 ---  rock
last2  01
history -----  0120
computer_move_name 2 -------- rock
user_move_name 2 -------- paper
y  0.7670835784845225
computer_move_name 1 ---  rock
last2  12
history -----  01201
computer_move_name 2 -------- rock
user_move_name 2 -------- scissors
y  0.8475755802234489
computer_move_name 1 ---  paper
last2  20
history -----  012012
last_chars ---------------- 012
pattern : --- 0
hi
computer_move_name 2 -------- paper
user_move_name 2 -------- rock
y